In [1]:
from stem import Signal
from stem.control import Controller
import requests
import bs4
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
import time
import random 
from selenium.webdriver import ActionChains
import requests
import json
import matplotlib.pylab as plt
from pandas.io.json import json_normalize
import numpy as np
from sklearn.linear_model import  Lasso
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Scrapper utility

In [451]:
class Scrapper():
    
    def __init__(self): 
        
        executable = "geckodriver"
        self.driver = webdriver.Firefox(executable_path='/Users/faraday/Desktop/HackUPC2019/restbai/geckodriver')
            
    def switchIP(self):
        
        with Controller.from_port(port = 9051) as controller:
            controller.authenticate()
            controller.signal(Signal.NEWNYM)
            

    # get a new selenium webdriver with tor as the proxy
    def my_proxy(self,PROXY_HOST,PROXY_PORT):
        
        fp = webdriver.FirefoxProfile()
        # Direct = 0, Manual = 1, PAC = 2, AUTODETECT = 4, SYSTEM = 5
        fp.set_preference("network.proxy.type", 1)
        fp.set_preference("network.proxy.socks",PROXY_HOST)
        fp.set_preference("network.proxy.socks_port",int(PROXY_PORT))
        fp.update_preferences()
        options = Options()
        options.headless = True
        
        return webdriver.Firefox(executable_path = "../geckodriver", options=options, firefox_profile=fp)
    
        
    def get_Link(self,url):
        self.driver.get(url)
        return(self.driver)
        
    def get_Items(self,max_entries):
        lista = []
        i = 0
        while (i < max_entries):
            i += 1
            elements = self.driver.find_elements_by_class_name("_8ssblpx")
            for element in elements:
                image = element.find_element_by_class_name("_1i2fr3fi")
               # url_imatge = image.get_attribute("style").split('"')[1]
               # llista_imatges = []
               # while url_imatge not in llista_imatges:
               #     print("selecting...")
               #     llista_imatges.append(url_imatge)
               #   ActionChains(self.driver).move_to_element(image).perform()
               #     swipe_image = element.find_element_by_class_name("_1rftspj9").click()
               #     time.sleep(1)
               #     image = element.find_element_by_class_name("_1i2fr3fi")
               #     url_imatge = image.get_attribute("style").split('"')[1]
               #     print(url_imatge)
               # print("Success!")
               # time.sleep(100)
                preu =  element.find_element_by_class_name("_1p3joamp").text
                try:
                    puntuacio = element.find_element_by_class_name("_tghtxy2").text
                except:
                    puntuacio = "none"
                tipus = element.find_element_by_class_name("_1xxanas2").text
                descripcio = element.find_element_by_class_name("_1dss1omb").text
                items = element.find_element_by_class_name("_1jlnvra2").text

                print(image.get_attribute("style").split('"')[1])
                print(preu)
                #print(puntuacio.text)
                #print(tipus.text)
                #print(descripcio.text)
                #print(items.text)

                data = (tipus,descripcio,items,preu,puntuacio,image.get_attribute("style").split('"')[1])
                lista.append(data)

            try:
                self.driver.find_element_by_class_name("optanon-allow-all.accept-cookies-button").click()
                time.sleep(5)
                self.driver.find_element_by_css_selector("._r4n1gzb > a:nth-child(1) > div:nth-child(1) > svg:nth-child(1)").click()
                time.sleep(5)
            except:
                try:
                    self.driver.find_element_by_css_selector("._r4n1gzb > a:nth-child(1) > div:nth-child(1) > svg:nth-child(1)").click()
                    time.sleep(5)
                except:
                    break

            
        return(lista)
    
    
            
    def next_url(self):
        self.driver.find_element_by_css_selector("._r4n1gzb > a:nth-child(1) > div:nth-child(1) > svg:nth-child(1)").click()
        return(self.driver.current_url)
        
    def list_urls(self):
        self.driver.find_element_by_class_name("optanon-allow-all.accept-cookies-button").click()
        links_list=[]
        for element in self.driver.find_elements_by_class_name("_15tommw"):
            links_list.append(element.get_attribute('href'))
        return(links_list)
        
    def close(self):
        self.driver.close()
        print("Successfully closed!")
        
    def take_pics(self,url):
        list_images = []
        image="none"
        self.driver.get(url)
        print(self.driver.current_url)
        #time.sleep(10)
        try:
            self.driver.find_element_by_xpath("//div[@class='_1f4354o4']").click()
        except:
            try:
                print("retrying..")
                time.sleep(1)
                self.driver.find_element_by_xpath("//div[@class='_1f4354o4']").click()
            except:
                try:
                    print("again...")
                    time.sleep(1)
                    self.driver.find_element_by_xpath("//div[@class='_1f4354o4']").click()
                except:
                    return ("Fail")
                    
        
        while image not in list_images:
            image = self.driver.find_element_by_class_name('_17vo0vv').get_attribute('src')
            list_images.append(image)
            siguiente = self.driver.find_element_by_xpath("//div[@class='_qmcx2d']").click()
        return(list_images)
        

In [452]:
airbnb = Scrapper()
airbnb.get_Link("https://www.airbnb.es/s/Barcelona/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&source=mc_search_bar&search_type=unknown&screen_size=large&checkin=2019-12-18&checkout=2019-12-20&hide_dates_and_guests_filters=true&room_types%5B%5D=Private%20room&ne_lat=41.397639736511465&ne_lng=2.169729774083436&sw_lat=41.36425403939052&sw_lng=2.120359961023979&zoom=14&search_by_map=true")
data = airbnb.get_Items(30)
df = pd.DataFrame(data)
df.columns = ['Tipo','Descripcion','Items','Precio','Valoracion','Imagen']

<selenium.webdriver.firefox.webdriver.WebDriver (session="e9f1e28f-7a22-024c-afd7-ba01735c8684")>

In [457]:
df.to_csv("Imatges.csv")

# API usage 

In [11]:
def get_features_v3(image):
    #define payload
    payload = {
        # Add your client key
        'client_key': client,
        'model_id': 're_features_v3',
        # Add the image URL you want to process
        'image_url': image
    }
    
    # Make the segmentation request
    response = requests.get(url, params=payload)
    
    # The response is formatted in JSON
    json_response = response.json()
    
    # if list is empty, it does not find anything
    if not json_response['response']['solutions']['re_features_v3']['detections']:
        items=[]
    
    else:
        # get items of json
        df_aux = pd.DataFrame.from_dict(json_normalize(json_response['response']['solutions']['re_features_v3']['detections']), orient='columns')
        items=df_aux['label'].tolist()
    
    return items


def get_room_type(image):
    #define payload
    payload = {
        # Add your client key
        'client_key': client,
        'model_id': 'real_estate_global_v2',
        # Add the image URL you want to process
        'image_url': image
    }
    
    # Make the segmentation request
    response = requests.get(url, params=payload)
    
    # The response is formatted in JSON
    json_response = response.json()
    
    # get items of json
    item=json_response['response']['solutions']['real_estate_global_v2']['top_prediction']['label']
    probab=json_response['response']['solutions']['real_estate_global_v2']['top_prediction']['confidence']

    return item,probab

def kitchen_condition(image):
    #define payload
    payload = {
        # Add your client key
        'client_key': client,
        'model_id': 're_cond_kitchen',
        # Add the image URL you want to process
        'image_url': image
    }
    
    # Make the segmentation request
    response = requests.get(url, params=payload)
    
    # The response is formatted in JSON
    json_response = response.json()
    
    # get items of json
    #item=json_response['response']['solutions']['real_estate_global_v2']['top_prediction']['label']
    #probab=json_response['response']['solutions']['real_estate_global_v2']['top_prediction']['confidence']

    return json_response#item,probab

def fill_row(df,items,l):
    
    #define zeros    
    zeros=np.zeros(len(columns))

    # if items is in list of columns add dataframe row
    for i in range(len(items)):
        for j in range(len(df.columns.tolist())):
            if items[i]==df.columns.tolist()[j]:
                #print(i,j)
                zeros[j]=1

    #add row
    df.loc[l]=zeros
                        
    return df        

In [12]:
url = 'https://api-eu.restb.ai/vision/v2/predict'
client='5a978ad15a5d7426f27c8e6cb7f8cc17332739e39ff0164b10f30a40a00c509b'
columns=['beamed_ceiling', 'carpet', 'coffered_ceiling', 'exposed_bricks', 'fireplace', 'french_doors', 'hardwood_floor', 'high_ceiling', 'kitchen_bar', 'kitchen_island', 'natural_light', 'notable_chandelier', 'skylight', 'stainless_steel', 'tile_floor', 'vaulted_ceiling',
         'ceiling_fan', 'central_ac', 'deck', 'dock', 'fire_pit', 'hot_tub', 'lawn', 'mountain_view', 'outdoor_kitchen', 'outdoor_living_space', 'pergola', 'pool', 'water_view']


In [13]:
df_images=pd.read_csv('dataset.csv')
df_images['Target']=df_images.apply(lambda x : float(x['Precio'].split("€")[-2].split("\n")[-1]),axis=1)
df_images['Valoracion']=df_images['Valoracion'].replace('none',np.NaN)
df_images['Valoracion']=df_images['Valoracion'].astype(float)

In [14]:
items_list=[]
df = pd.DataFrame(columns=columns)
zeros=np.zeros(len(columns))
k=0

for image in df_images['Imagen'].tolist():
    item=get_features_v3(image)
    items_list.append(item)
    print('{:.2f}% : {} of {}'.format(k/len(df_images['Imagen'].tolist())*100,k,len(df_images['Imagen'].tolist())))
    k +=1
    
for k in range(len(items_list)):
    df=fill_row(df,items_list[k],k)
    
 
df['Target_Price']=df_images['Target']
df['Target_Valoration']=df_images['Valoracion']       

0.00% : 0 of 162
0.62% : 1 of 162
1.23% : 2 of 162
1.85% : 3 of 162
2.47% : 4 of 162
3.09% : 5 of 162
3.70% : 6 of 162
4.32% : 7 of 162
4.94% : 8 of 162
5.56% : 9 of 162
6.17% : 10 of 162
6.79% : 11 of 162
7.41% : 12 of 162
8.02% : 13 of 162
8.64% : 14 of 162
9.26% : 15 of 162
9.88% : 16 of 162
10.49% : 17 of 162
11.11% : 18 of 162
11.73% : 19 of 162
12.35% : 20 of 162
12.96% : 21 of 162
13.58% : 22 of 162
14.20% : 23 of 162
14.81% : 24 of 162
15.43% : 25 of 162
16.05% : 26 of 162
16.67% : 27 of 162
17.28% : 28 of 162
17.90% : 29 of 162
18.52% : 30 of 162
19.14% : 31 of 162
19.75% : 32 of 162
20.37% : 33 of 162
20.99% : 34 of 162
21.60% : 35 of 162
22.22% : 36 of 162
22.84% : 37 of 162
23.46% : 38 of 162
24.07% : 39 of 162
24.69% : 40 of 162
25.31% : 41 of 162
25.93% : 42 of 162
26.54% : 43 of 162
27.16% : 44 of 162
27.78% : 45 of 162
28.40% : 46 of 162
29.01% : 47 of 162
29.63% : 48 of 162
30.25% : 49 of 162
30.86% : 50 of 162
31.48% : 51 of 162
32.10% : 52 of 162
32.72% : 53 of 162
3

# Machine learning 

### XGB model

In [15]:
Y_train=df['Target_Price'].tolist()
X_train=df.drop(['Target_Price','Target_Valoration'],axis=1)
X_train, X_val, y_train, y_val= train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

In [16]:
xgb_model = XGBRegressor()#objective='squarederror') 

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

grid = GridSearchCV(xgb_model, params, cv=3, refit=True)
grid.fit(X_train,y_train)
grid.best_score_
#grid.best_estimator_.feature_importance

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-0.005896082246682727

In [17]:
y_xgb=grid.predict(X_val)
print("Mean squared error for validation test is {}".format((mean_squared_error(y_true=y_val, y_pred=y_xgb))**0.5))

Mean squared error for validation test is 10.327419915622627
